In [10]:
import pandas as pd
import addfips
from urllib.request import urlopen
import json
import plotly.graph_objects as go

In [11]:
# DATA
df1 = pd.DataFrame({
    'state': ['California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California'],
    'county': ['Sonoma', 'Napa', 'Solano', 'Marin', 'Contra Costa', 'Alameda', 'Santa Clara', 'San Mateo', 'San Francisco'],
    'income': [102840, 108970, 99994, 142785, 125727, 126240, 159674, 156000, 141446]
})

df2 = pd.DataFrame({
    'state': ['California', 'California', 'California', 'California', 'California', 'California', 'California', 'California', 'California'],
    'county': ['Sonoma', 'Napa', 'Solano', 'Marin', 'Contra Costa', 'Alameda', 'Santa Clara', 'San Mateo', 'San Francisco'],
    'aqi': [8.3, 10.3, 12.4, 8.6, 11.1, 11.8, 12.5, 9.8, 10.5]
})

In [12]:
# Adding FIPS codes
# Requires the addfips package to be installed

af = addfips.AddFIPS()

for df in [df1, df2]:
    df['FIPS'] = df.apply(lambda row: af.get_county_fips(row['county'], row['state']), axis=1)

cols1 = ['FIPS'] + [col for col in df1.columns if col != 'FIPS']
df1 = df1[cols1]

cols2 = ['FIPS'] + [col for col in df2.columns if col != 'FIPS']
df2 = df2[cols2]

In [13]:
# GeoJASON data for counties
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
# Choropleth
fig = go.Figure()

# Income trace
fig.add_trace(go.Choropleth(
    geojson=counties,
    locations=df1['FIPS'],
    z=df1['income'],
    colorscale='Greens',
    zmin=df1['income'].min()/2,
    zmax=df1['income'].max(),
    colorbar_title="Income",
    marker_line_width=0,
    visible=True,
    name="Income"
))

# AQI trace
fig.add_trace(go.Choropleth(
    geojson=counties,
    locations=df2['FIPS'],
    z=df2['aqi'],
    colorscale='Reds',
    zmin=0,
    zmax=30,
    colorbar_title="AQI",
    marker_line_width=0,
    visible=False,
    name="AQI"
))

########################################################################
# Zoom!
fig.update_geos(fitbounds="locations", visible=False)
########################################################################

# Layout and Dropdown menu
fig.update_layout(
    title_text="Bay Area County-Level Income and Air Quality (AQI)",
    autosize=False,
    geo_scope='usa',
    updatemenus=[dict(
        active=0,
        buttons=list([
            dict(label="Income",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"coloraxis": {"colorbar": {"title": "Income"}},
                        "title": "Bay Area County-Level Income"}]),
            dict(label="AQI",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"coloraxis": {"colorbar": {"title": "AQI"}},
                        "title": "Bay Area County-Level Air Quality Index"}])
        ]),
        direction="down",
        showactive=True
    )]
)

fig.show()

In [15]:
fig.write_html("BayArea_income_aqi.html")